In [2]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import os.path as osp
import time
from torchmetrics.functional import retrieval_average_precision,retrieval_normalized_dcg
import pandas as pd
def vectorize_sort(x, permutation):
    # Order tensor by indecis
    d1, d2 = x.size()
    ret = x[
        torch.arange(d1).unsqueeze(1).repeat((1, d2)).flatten(),
        permutation.flatten(),
    ].view(d1, d2)
    return ret


In [3]:
path = '/Users/kbibas/code/fbt_research/outputs/train_cycle_with_clf_Clothing_Shoes_and_Jewelry_20220708_192523'
src = torch.load(osp.join(path,'src.pth'))
src_fbt = torch.load(osp.join(path,'src_fbt.pth'))
asin_src = torch.load(osp.join(path,'asin_src.pth'))
candidates_order  = torch.load(osp.join(path,'candidates_order.pth'))
category_int_src_test = torch.load(osp.join(path,'category_int_src_test.pth'))
category_int_pos_test = torch.load(osp.join(path,'category_int_pos_test.pth'))
hot_label_test = torch.load(osp.join(path,'hot_label_test.pth'))
set_name = torch.load(osp.join(path,'set_name.pth'))

FileNotFoundError: [Errno 2] No such file or directory: '/Users/kbibas/code/fbt_research/outputs/train_cycle_with_clf_Clothing_Shoes_and_Jewelry_20220708_192523/src.pth'

In [5]:
src_fbt_test = src_fbt[set_name=='test']

In [8]:
# Find distance of the candidates
t2 = time.time()
dists = torch.cdist(src_fbt_test, candidates_order, p=2)
probs = torch.softmax(-dists, axis=-1)

# Calculate mAP
t2 = time.time()
rmap_val = retrieval_average_precision(probs, hot_label_test).item()
ndcg_val =  retrieval_normalized_dcg(probs, hot_label_test).item()
print(f'{ndcg_val=:.5f} {rmap_val=:.5f}')

# TopK
t2 = time.time()
_, sort_idxs = torch.sort(dists, dim=-1)
is_true = vectorize_sort(hot_label_test, sort_idxs)
topk_d = {}
top_k = [1,3,5,10]
for k in top_k:
    topk = torch.any(is_true[:, :k], axis=-1).float().mean().item()
print(f'{topk_d}')

ndcg_val=0.65279 rmap_val=0.00952


KeyboardInterrupt: 